# 5 - Interpretação e Avaliação dos Resultados

Última etapa do processo KDD: interpretação das classificações geradas no notebook de mineração, análise dos critérios mais frequentes e cruzamento com indicadores da Lei nº 14.133/2021.

## Metodologia de Interpretação

A análise dos resultados segue três eixos principais:

1. **Distribuição das Classificações**: Quantificação por categoria de risco
2. **Análise por Critério**: Identificação dos padrões mais frequentes
3. **Cruzamento Normativo**: Alinhamento com dispositivos da Lei 14.133/2021

**Referências:**
- Fayyad et al. (1996): Interpretação de padrões no processo KDD
- Brasil (2021): Lei 14.133/2021 como framework de validação


## 1. Configuração

Carregamos bibliotecas de análise (`pandas`, `numpy`) e definimos opções de visualização para facilitar a leitura dos relatórios.


In [1]:

import pandas as pd
import numpy as np
from pathlib import Path

pd.options.display.max_columns = 40
pd.options.display.float_format = lambda x: f"{x:,.2f}"

exports_dir = Path('exports')



## 2. Carregamento do dataset minerado

Utilizamos `licitacoes_com_score.csv`, exportado no notebook de mineração. Ele já inclui:
- atributos originais e derivados;
- `score_anomalia` (faixa 0–1);
- `nivel_risco` e `flag_anomalia` (percentil 90).


In [2]:

dataset_path = exports_dir / '3_licitacoes_com_score.csv'

if not dataset_path.exists():
    raise FileNotFoundError('Execute `mineracao.ipynb` antes de prosseguir.')

df = pd.read_csv(dataset_path, sep=';')
print(f'Registros carregados: {len(df):,}')
df.head(3)


Registros carregados: 836


,numprocesso,ano,modalidade,situacao,objeto,orgao,criteriojulgamento,valor_estimado,valor_ou_desconto_homologado,dataabertura,datahomologacao,arquivo,objeto_normalizado,diferenca_valor,variacao_percentual,proporcao_homologado_estimado,flag_homologado_acima_estimado,flag_houve_desconto,lead_time_dias,criterio_menor_preco,flag_situacao_certame,flag_informatica,flag_certame_em_andamento,flag_sem_desconto,flag_nao_menor_preco,classificacao,justificativa_anomalia,flag_anomalia
0,4,2025,Audiência Pública,Fase Certame,Audiência Pública que visa subsidiar o process...,SEAP-Secretaria de Estado da Administração e d...,Menor Preço,0.00,NaN,2025-07-08,NaN,TB_LICITACOES-2025.csv,audiência pública que visa subsidiar o process...,0.00,0.00,0.00,False,True,0.00,True,True,True,True,False,False,Regular,Nenhum critério de anomalia identificado,False
1,10,2025,Audiência Pública,Fase Certame,"relativo a contratação de empresa, ou consórci...",SGSD-Superintendência Geral de Governança de S...,Menor Preço,0.00,NaN,2025-09-11,NaN,TB_LICITACOES-2025.csv,"relativo a contratação de empresa, ou consórci...",0.00,0.00,0.00,False,True,0.00,True,True,True,True,False,False,Regular,Nenhum critério de anomalia identificado,False
2,23,2025,Chamamento Público,Fase Certame,Credenciamento de Permissionários para exercer...,DER-Departamento de Estradas de Rodagem do Est...,-,0.00,NaN,2025-09-17,NaN,TB_LICITACOES-2025.csv,credenciamento de permissionários para exercer...,0.00,0.00,0.00,False,True,0.00,False,True,True,True,False,True,Regular,Nenhum critério de anomalia identificado,False


## 3. Distribuição das Classificações

Análise quantitativa das três categorias de risco definidas pelo sistema de regras.

In [3]:
# Distribuição por categoria
distribuicao = df['classificacao'].value_counts().sort_index()
distribuicao_pct = (distribuicao / len(df) * 100).round(2)

resumo = pd.DataFrame({
    'Quantidade': distribuicao,
    'Percentual': distribuicao_pct.apply(lambda x: f"{x:.2f}%")
})

print("="*60)
print("DISTRIBUIÇÃO DAS CLASSIFICAÇÕES")
print("="*60)
print(resumo)
print(f"\nTotal analisado: {len(df):,} licitações")
print(f"Anomalias detectadas: {df['flag_anomalia'].sum():,} ({df['flag_anomalia'].mean()*100:.1f}%)")

DISTRIBUIÇÃO DAS CLASSIFICAÇÕES
                   Quantidade Percentual
classificacao                           
Anomalia Crítica            3      0.36%
Regular                   799     95.57%
Suspeita Moderada          34      4.07%

Total analisado: 836 licitações
Anomalias detectadas: 37 (4.4%)


## 4. Análise por Critério de Anomalia

Identificação dos critérios mais frequentemente violados nas licitações classificadas como anômalas.

In [4]:
# Filtrar apenas anomalias
anomalias_df = df[df['flag_anomalia']].copy()

print(f"\nAnálise de {len(anomalias_df):,} licitações anômalas\n")
print("="*60)

# Contar critérios nas justificativas
criterios = {
    'Sobrepreço': anomalias_df['justificativa_anomalia'].str.contains('Sobrepreço', case=False, na=False).sum(),
    'Ausência de Competitividade': anomalias_df['justificativa_anomalia'].str.contains('Ausência de competitividade', case=False, na=False).sum(),
    'Prazo Excessivo': anomalias_df['justificativa_anomalia'].str.contains('Prazo excessivo', case=False, na=False).sum(),
    'Múltiplos Sinais': anomalias_df['justificativa_anomalia'].str.contains('Múltiplos sinais', case=False, na=False).sum()
}

criterios_df = pd.DataFrame.from_dict(criterios, orient='index', columns=['Ocorrências'])
criterios_df['Percentual'] = (criterios_df['Ocorrências'] / len(anomalias_df) * 100).round(2).apply(lambda x: f"{x:.1f}%")

print("CRITÉRIOS MAIS FREQUENTES NAS ANOMALIAS:")
print(criterios_df.sort_values('Ocorrências', ascending=False))


Análise de 37 licitações anômalas

CRITÉRIOS MAIS FREQUENTES NAS ANOMALIAS:
                             Ocorrências Percentual
Prazo Excessivo                       31      83.8%
Ausência de Competitividade            4      10.8%
Sobrepreço                             2       5.4%
Múltiplos Sinais                       1       2.7%


In [5]:
# Top 20 anomalias críticas
top_criticas = (
    df[df['classificacao'] == 'Anomalia Crítica']
    .sort_values(by='valor_estimado', ascending=False)
    .head(20)
)

colunas_exibir = [
    'numprocesso', 'ano', 'modalidade', 'orgao',
    'valor_estimado', 'valor_ou_desconto_homologado',
    'classificacao', 'justificativa_anomalia'
]

print("="*80)
print("TOP 20 ANOMALIAS CRÍTICAS (por valor estimado)")
print("="*80)
display(top_criticas[colunas_exibir])

TOP 20 ANOMALIAS CRÍTICAS (por valor estimado)


,numprocesso,ano,modalidade,orgao,valor_estimado,valor_ou_desconto_homologado,classificacao,justificativa_anomalia
459,698,2023,Pregão Eletrônico,EPR-Serviço Social Autônomo E-PARANA COMUNICACAO,"13,829,739.00","21,971,900.00",Anomalia Crítica,Sobrepreço de 58.9% (acima do limite de 10.0%)
511,1342,2023,Pregão Eletrônico,LOTTOPAR-Loteria do Estado do Paraná,"6,060,000.00","9,730,000.00",Anomalia Crítica,Sobrepreço de 60.6% (acima do limite de 10.0%)
620,212,2022,Concorrência Pública,UNIOESTE CASC-Universidade Estadual do Oeste d...,"5,550,000.00","5,900,000.00",Anomalia Crítica,Ausência de competitividade: critério diferent...



## 5. Sinais relacionados à Lei nº 14.133/2021

A Lei nº 14.133 enfatiza planejamento, eficiência e julgamento adequado. Cruzamos as anomalias com indicadores associados a esses princípios:

- Referência explícita à lei (modalidade contendo "14.133");
- Uso do critério `Menor Preço`;
- Existência de desconto (
  \( 	ext{flag\_sem\_desconto} = 1 \iff 	ext{proporcao\_homologado\_estimado} \geq 1 \));
- Situação em "Fase Certame" (processo em andamento).

A tabela a seguir mostra, para cada combinação, a contagem de licitações e a proporção de anomalias.


In [6]:
# Identificar licitações sob nova lei
df['modalidade_ref_14133'] = df['modalidade'].str.contains('14.133', case=False, na=False)

print("="*60)
print("ANÁLISE: IMPACTO DA LEI 14.133/2021")
print("="*60)

# Comparar taxa de anomalias entre leis
comparacao_lei = df.groupby('modalidade_ref_14133').agg({
    'flag_anomalia': ['sum', 'count', 'mean']
}).round(4)

comparacao_lei.columns = ['Total_Anomalias', 'Total_Licitacoes', 'Taxa_Anomalia']
comparacao_lei['Taxa_Anomalia'] = (comparacao_lei['Taxa_Anomalia'] * 100).round(2).apply(lambda x: f"{x}%")

comparacao_lei.index = ['Lei 8.666 (antiga)', 'Lei 14.133 (nova)']
print(comparacao_lei)

# Análise por critério de julgamento
print("\n" + "="*60)
print("ANÁLISE POR CRITÉRIO DE JULGAMENTO")
print("="*60)

criterio_analise = df.groupby('criterio_menor_preco').agg({
    'flag_anomalia': ['sum', 'count', 'mean']
}).round(4)

criterio_analise.columns = ['Total_Anomalias', 'Total_Licitacoes', 'Taxa_Anomalia']
criterio_analise['Taxa_Anomalia'] = (criterio_analise['Taxa_Anomalia'] * 100).round(2).apply(lambda x: f"{x}%")
criterio_analise.index = ['Outros Critérios', 'Menor Preço']

print(criterio_analise)

ANÁLISE: IMPACTO DA LEI 14.133/2021
                    Total_Anomalias  Total_Licitacoes Taxa_Anomalia
Lei 8.666 (antiga)               25               460         5.43%
Lei 14.133 (nova)                12               376         3.19%

ANÁLISE POR CRITÉRIO DE JULGAMENTO
                  Total_Anomalias  Total_Licitacoes Taxa_Anomalia
Outros Critérios               10                67        14.93%
Menor Preço                    27               769         3.51%
